In [1]:
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.applications import VGG16
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import IPython

In [2]:
!pip install keras-tuner
!pip install hiplot

In [4]:

import kerastuner as kt

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [8]:
validation_dir="D:\params_datasets\cleaned_data_4_folders\\new_automated_folders\\traffic_dataset_cleaned\\validation"


In [9]:
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150,150),
                                                        batch_size=64,
                                                    class_mode='categorical')

Found 20000 images belonging to 4 classes.


In [10]:
print("validation done")

validation done


In [11]:
import tensorflow as tf
def model_builder(hp):
  #------Tuning for Image Augumentation------
  global train_datagen
  train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   zoom_range=hp.Float('zoom_range',
                                                        min_value=0.0,
                                                        max_value=0.5,
                                                        default=0.05,
                                                        step=0.05
                                                      ),
                                   width_shift_range=hp.Float('width_shift_range',
                                                        min_value=0.0,
                                                        max_value=0.5,
                                                        default=0.05,
                                                        step=0.05
                                                      ),
                                   height_shift_range=hp.Float('height_shift_range',
                                                        min_value=0.0,
                                                        max_value=0.5,
                                                        default=0.05,
                                                        step=0.05
                                                      ),
                                   shear_range=hp.Float('shear_range',
                                                        min_value=0.0,
                                                        max_value=0.5,
                                                        default=0.05,
                                                        step=0.05
                                                      ),
                                   vertical_flip=hp.Choice('vertical_flip',
                                                            values=[True,False],
                                                            default=True),
                                   fill_mode="nearest") 

  #------Tuning for number of 2D-CNN blocks-------

  num_block = hp.Int('num_block', min_value=2, max_value=10, step=1)

  num_filters=[]
  for i in range(num_block):
    num_filters.append(hp.Int(f'num_filters_{str(i)}', min_value=32, max_value=128, step=32))

  dropout=[]
  for j in range(num_block):
    dropout.append(hp.Float(f'dropout_{str(j)}',min_value=0.0,max_value=0.5,default=0.25,step=0.05))

  # num_filters = hp.Int('num_filters', min_value=32, max_value=128, step=32)
  model = keras.Sequential()
  model.add(
            tf.keras.layers.Conv2D(
                filters=num_filters[0],
                kernel_size=3,
                activation='relu',
                padding='same',
                input_shape=(150,150,3)
            )
        )
  model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
  model.add(
            tf.keras.layers.Dropout(
                rate=dropout[0]
            )
        )
  for i in range(1,num_block):
    model.add(
            tf.keras.layers.Conv2D(
                filters=num_filters[i],
                kernel_size=3,
                activation='relu',
                padding='same',
            )
        )
    # model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(
            tf.keras.layers.Dropout(
                rate=dropout[i]
            )
        )
    
        #flattening


  model.add(tf.keras.layers.Flatten())


  #------tuning for number of Dense layers---------
  dence_block = hp.Int('dence_block', min_value=2, max_value=6, step=1)
  layer_Neurons=[]
  dense_activation=[]
  for i in range(dence_block):
    layer_Neurons.append(hp.Int(f'layer_Neurons_{str(i)}',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ))
    dense_activation.append(hp.Choice(f'dense_activation_{str(i)}',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                ))
        #dense_layer_1
  for i in range(dence_block):
    model.add(
            tf.keras.layers.Dense(
                units=layer_Neurons[i],
                activation=dense_activation[i]
            )
        )  
  model.add(tf.keras.layers.Dense(4, activation='softmax'))
#------tuning for Learning rate---------
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(loss='categorical_crossentropy',
                optimizer=optimizers.Adam(learning_rate=hp_learning_rate),
                metrics=['accuracy'])
  return model

In [12]:
print("modelling done")

modelling done


In [13]:
from kerastuner.tuners import RandomSearch
tuner = RandomSearch(
    model_builder,
    objective='val_accuracy',
    max_trials=25,
    executions_per_trial=3,
    directory="D:\params_datasets\cleaned_data_4_folders\\new_automated_folders\\traffic_dataset_cleaned\\tuning_results",
    project_name='logs_and_checkpoints')


In [16]:
train_dir="D:\params_datasets\cleaned_data_4_folders\\new_automated_folders\\traffic_dataset_cleaned\\train"
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150,150),
                                                    batch_size=64,
                                                    class_mode='categorical')

Found 391486 images belonging to 4 classes.


In [ ]:
# buildModel(hp)
# model = tf.keras.models.load_model('ModelCheckPoint\ckpt.hdf5')
# do this for resuming training

In [17]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [18]:
checkpoint_path="D:\params_datasets\cleaned_data_4_folders\\new_automated_folders\\traffic_dataset_cleaned\\tuning_results\checkpoint_path\\model{epoch:08d}.hdf5"
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_best_only=True, save_weights_only=False, verbose=1),
    tf.keras.callbacks.TensorBoard(log_dir="D:\params_datasets\cleaned_data_4_folders\\new_automated_folders\\traffic_dataset_cleaned\\tuning_results\\tensorboard_log_dir"),
]

In [ ]:
tuner.search(train_generator,
             steps_per_epoch=391486// 3913,
             epochs=30,
             validation_data=validation_generator,
             validation_steps=20000 // 1333,
             verbose=1,
             callbacks=my_callbacks)